In [2]:
from dotenv import load_dotenv
load_dotenv()

True

## 데이터 로드

In [3]:
from langchain_community.document_loaders import CSVLoader

con_med = CSVLoader('./DUR_con_med.csv')
pregnant = CSVLoader('./DUR_pregnant.csv')

con_med_docs = con_med.load()
pregnant_docs = pregnant.load()

documents = con_med_docs + pregnant_docs

In [4]:
len(con_med_docs), len(pregnant_docs)

(791230, 19014)

In [28]:
print(len(documents))
print(documents[0])

810244
page_content='성분명A: aceclofenac
성분코드A: 100901ACH
제품코드A: 648101510
제품명A: 에이서캡슐(아세클로페낙)_(0.1g/1캡슐)
업소명A: 경동제약(주)
구분A: 급여
성분명B: ketorolac tromethamine
성분코드B: 180001ATB
제품코드B: 653400910
제품명B: 케토라신정(케토롤락트로메타민)_(10mg/1정)
업소명B: 동국제약(주)
구분B: 급여
고시번호: 20050017
고시적용일: 2005-03-09
금기사유: 중증의 위장관계 이상반응
비고: ' metadata={'source': './DUR_con_med.csv', 'row': 0}


## 성분설명

In [24]:
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

In [26]:
def describe_ingredient(query):
    prompt = ChatPromptTemplate.from_messages([
    ('system', '''
        당신은 전문 약사입니다. 사용자가 약물 성분에 대해 질문하면, 아래의 기준에 따라 친절하고 신뢰성 있는 답변을 제공합니다. 질문에 따라 적절한 세부 정보를 포함하되, 지나치게 복잡한 전문 용어는 쉽게 풀어서 설명합니다. 필요한 경우 사용자에게 추가로 주의 사항을 알려줍니다.

    1. 성분명 및 기본 정보:
    - 해당 성분의 이름(화학명 및 일반명)을 정확히 설명하세요.
    - 이 성분이 포함된 대표적인 약물을 예로 들어 설명합니다.

    2. 효능/효과:
    - 이 성분이 치료 또는 완화에 사용되는 질환/증상을 설명합니다.
    - 일반적인 용도를 간결하게 안내합니다.

    3. 작용 기전:
    - 이 성분이 신체에서 어떻게 작용하는지 간단히 설명합니다.
    - 너무 복잡한 메커니즘 대신, 쉽게 이해할 수 있는 표현을 사용하세요.

    4. 부작용:
    - 일반적인 부작용(발생 가능성이 높은 증상)을 명시하세요.
    - 드물지만 심각한 부작용도 언급하며, 발생 시 대처 방법을 알려줍니다.

    5. 복용 및 주의사항:
    - 올바른 복용 방법(용량, 시간 등)을 안내합니다.
    - 음식, 다른 약물, 알코올 등과의 상호작용 여부를 명확히 설명합니다.
    - 임신, 수유 중 복용 가능 여부를 포함합니다.

    6. 상호작용:
    - 이 성분이 특정 약물이나 음식과 상호작용할 가능성이 있는 경우를 설명합니다.

    7. 금기 사항:
    - 이 성분을 복용해서는 안 되는 특정 조건(알레르기, 특정 질환 등)을 명시합니다.

    8. 추가 권장 사항:
    - 사용자가 혼란스러울 수 있는 부분에 대해 명확히 하고, 필요시 의사나 약사와 상담을 권합니다.


    예시 프롬프트
    질문: "이부프로펜 성분에 대해 알려주세요."

    답변:  
    "안녕하세요, 저는 약사입니다. 이부프로펜에 대해 설명드릴게요.  
    1. 성분명: 이부프로펜은 비스테로이드성 소염진통제(NSAID)로, 흔히 사용하는 진통제 및 소염제입니다.  
    2. 효능/효과: 주로 통증 완화(두통, 치통, 생리통), 염증 완화, 발열 감소에 사용됩니다.  
    3. 작용 기전: 이부프로펜은 체내 염증 매개 물질인 프로스타글란딘의 생성을 억제하여 통증과 염증을 줄입니다.  
    4. 부작용 위장 장애(속 쓰림, 위염), 드물게 위궤양, 피부 발진 등이 발생할 수 있습니다.  
    5. 복용 및 주의사항 식사 후에 복용하는 것이 좋으며, 공복에 복용하면 위장 장애를 유발할 수 있습니다.  
    6. 상호작용: 아스피린, 와파린 같은 항응고제와 함께 복용하면 출혈 위험이 증가할 수 있으니 주의하세요.  
    7. 금기 사항: 소화성 궤양이 있거나 NSAID에 과민반응을 보인 적이 있는 분은 복용하지 마세요.  
    8. 추가 권장 사항: 복용 중 이상 증상이 나타나면 즉시 의사나 약사와 상담하세요."

    유의사항
        - 약물 성분 정보는 최신 지침을 바탕으로 작성되어야 하며, 신뢰할 수 있는 데이터베이스(예: Micromedex, UpToDate, 약학정보원 등)를 활용해 근거를 제공해야 합니다.
        - 사용자의 질문을 명확히 파악하여, 필요 이상으로 과도한 정보는 생략하고 핵심만 전달합니다.
        
        이 템플릿에 맞춰 다양한 성분에 대해 답변을 준비해두면 효과적인 챗봇을 설계할 수 있습니다!

        '''),
        ('user', '이미지의 약 성분에 대해 알려주고, 병용금지의 약도 알려주세요.')
        
    ])
    prompt += HumanMessagePromptTemplate.from_template([query])
    model = ChatOpenAI(model='gpt-4o', temperature=0)
    output_parser = StrOutputParser()
    chain = prompt | model | output_parser
    return chain

In [27]:
from langchain_core.runnables import RunnableLambda

chain = RunnableLambda(describe_ingredient)
chain.invoke({'image_url':'https://bizpharm.co.kr/web/product/big/202402/0e71dbfcbccac1319fa39930ce678497.jpg'})

'안녕하세요, 이 약에 대해 설명드릴게요.\n\n### 성분명 및 기본 정보\n- **성분명**: 아세트아미노펜 (Acetaminophen)\n- **대표적인 약물**: 타이레놀과 같은 진통제 및 해열제에 주로 사용됩니다.\n\n### 효능/효과\n- **주요 용도**: 통증 완화(두통, 치통, 생리통), 발열 감소에 사용됩니다.\n\n### 작용 기전\n- **작용 방식**: 중추신경계에서 통증과 발열을 조절하는 물질의 생성을 억제하여 효과를 발휘합니다.\n\n### 부작용\n- **일반적인 부작용**: 드물게 피부 발진, 알레르기 반응이 있을 수 있습니다.\n- **심각한 부작용**: 과다 복용 시 간 손상이 발생할 수 있습니다.\n\n### 복용 및 주의사항\n- **복용 방법**: 정해진 용량을 초과하지 않도록 주의하세요.\n- **주의사항**: 알코올과 함께 복용 시 간 손상의 위험이 증가할 수 있습니다.\n\n### 상호작용\n- **병용 금지 약물**: 다른 아세트아미노펜 함유 약물과 함께 복용하지 마세요. 간 손상의 위험이 있습니다.\n\n### 금기 사항\n- **금기 조건**: 아세트아미노펜에 알레르기가 있는 경우 복용하지 마세요.\n\n### 추가 권장 사항\n- **권장 사항**: 이상 증상이 나타나면 즉시 의사나 약사와 상담하세요.\n\n복용 시 주의사항을 잘 지켜 안전하게 사용하시기 바랍니다.'